File where using AST many many formulas are parsed.
<br>
Also the specified task was split every formula into parts that can be aggregated (hence no division or multiplication of counters are allowed).

In [30]:
with open('samples.txt') as f:
    formulas = f.read()
    
formulas = formulas.split('\n')

Sample of what we are dealing with

In [31]:
formulas[-3:]

['100* (1 -(1 -((RRC_CONN_STP_FAIL_AC_DL)/ (RRC_CONN_STP_ATT -RRC_CONN_SETUP_ATT_REPEAT -RRC_CONN_ACC_REL_CELL_RESEL +RRC_CONN_SETUP_COMP_AFT_DIR -RRC_CONN_STP_REJ_EMERG_CALL)))* (1 -((DL_DCH_SEL_HSDPA_POWER_(INT,BGR,STR)) /((RAB_STP_ATT_CS_VOICE) +(RAB_STP_ATT_CS_ CONV)+(RAB_STP_ATT_PS_(INTER,BACKG)))))) = 100* (1 -(1 -((M1001C732)/ (M1001C0 -M1001C242 -M1001C241 +M1001C259 -M1001C617)))*(1 -((M1002C521 +M1002C522 +M1002C602) /((M1001C66) +(M1001C67) +(M1001C71 +M1001C72)))))',
 '100* ((IRATHO.SuccRelocPrepOutCS/ IRATHO.AttRelocPrepOutCS)* IRATHO.SuccOutCS/ (IRATHO.AttOutCS))',
 '100* ((((IRATHO.SuccRelocPrepInCS/ IRATHO.AttRelocPrepInCS)* IRATHO.SuccInCS) +(IRATHO.SuccRelocPrepOutCS/ IRATHO.AttRelocPrepOutCS)* IRATHO.SuccOutCS +(IRATHO.SuccRelocPrepOutPS/ IRATHO.AttRelocPrepOutPS)* IRATHO.SuccOutPSUTRAN )/ ((IRATHO.AttInCS) +(IRATHO.AttOutCS) +(IRATHO.AttOutPSUTRAN)))']

In [3]:
import string
import ast
import re
import numpy as np

In [4]:
def prepare_formula(formula):
    fixed = formula.replace(' ', '').replace('.', '_')
    fixed = fixed.split('=')
    return fixed[-1]

In [14]:
def create_tree(formula):
    def recurse(node, temp):
        if isinstance(node, ast.BinOp):
            if isinstance(node.op, ast.Add) or isinstance(node.op, ast.Sub):
                temp.append(['('])
            if isinstance(node.op, ast.Div) or isinstance(node.op, ast.Mult):
                temp.append(['('])
            recurse(node.left, temp)
            recurse(node.op, temp)
            recurse(node.right, temp)
            if isinstance(node.op, ast.Div) or isinstance(node.op, ast.Mult):
                temp.append([')'])
            if isinstance(node.op, ast.Add) or isinstance(node.op, ast.Sub):
                temp.append([')'])
        elif isinstance(node, ast.Add):
            temp.append(['+'])
        elif isinstance(node, ast.Sub):
            temp.append(['-'])
        elif isinstance(node, ast.Mult):
            temp.append(['*'])
        elif isinstance(node, ast.Div):
            temp.append(['/'])
        elif isinstance(node, ast.Num):
            temp.append([str(node.n)])
        elif isinstance(node, ast.Name):
            temp.append([node.id])
        else:
            for child in ast.iter_child_nodes(node):
                recurse(child, temp)
                return temp
    temp = []        
    #expr = ast.parse(formula, mode='eval')
    if formula is not None:
        recurse(formula, temp)
    return temp

In [51]:
def clean_formula(formula):
    def recurs_clean(node, temp):
        if isinstance(node, ast.BinOp):

            if isinstance(node.op, ast.Div) or isinstance(node.op, ast.Mult):
                if not isinstance(node.left, ast.Num) and not isinstance(node.left, ast.Name):
                    temp.append(['('])
                    
            elif isinstance(node.op, ast.Sub):
                if '-' in ''.join([i[0] for i in create_tree(node.left)]):
                    pass
                elif '-' in ''.join([i[0] for i in create_tree(node.right)]):
                    temp.append(['('])
                else:
                    pass
                    
            recurs_clean(node.left, temp)
            
            if isinstance(node.op, ast.Div) or isinstance(node.op, ast.Mult):
                if not isinstance(node.left, ast.Num) and not isinstance(node.left, ast.Name):
                    temp.append([')'])

            recurs_clean(node.op, temp)

            if isinstance(node.op, ast.Div) or isinstance(node.op, ast.Mult):
                if not isinstance(node.right, ast.Num) and not isinstance(node.right, ast.Name):
                    temp.append(['('])
                    
            recurs_clean(node.right, temp)
            
            if isinstance(node.op, ast.Div) or isinstance(node.op, ast.Mult):
                if not isinstance(node.right, ast.Num) and not isinstance(node.right, ast.Name):
                    temp.append([')'])
                    
            elif isinstance(node.op, ast.Sub):
                if '-' in ''.join([i[0] for i in create_tree(node.left)]):
                    pass
                elif '-' in ''.join([i[0] for i in create_tree(node.right)]):
                    temp.append(['('])
                else:
                    pass
                    
        elif isinstance(node, ast.Add):
            temp.append(['+'])
        elif isinstance(node, ast.Sub):
            temp.append(['-'])
        elif isinstance(node, ast.Mult):
            temp.append(['*'])
        elif isinstance(node, ast.Div):
            temp.append(['/'])
        elif isinstance(node, ast.Num):
            temp.append([str(node.n)])
        elif isinstance(node, ast.Name):
            temp.append([node.id])
        else:
            for child in ast.iter_child_nodes(node):
                recurs_clean(child, temp)
                return temp

    return ''.join([i[0] for i in recurs_clean(ast.parse(formula, mode='eval'), [])])

In [52]:
clean_formula('a-b-(b-c)*h-c-h*(k-6)')

'a-b-(b-c)*h-c-h*(k-6)'

In [18]:
from termcolor import colored

In [26]:
def process(formula):
    def check_div_mult(node, alias_dict):
        exp = ''.join([i[0] for i in create_tree(node)])
        if isinstance(node, ast.BinOp):
            if '/' in exp:
                for child in ast.iter_child_nodes(node):  
                    check_div_mult(child, alias_dict)
            elif '*' in exp and isinstance(node.left, ast.Num) or isinstance(node.right, ast.Num):
                alias_dict[exp]=1
            else:
                alias_dict[exp]=1
        elif isinstance(node, ast.Name):
            alias_dict[exp]= 1
        elif isinstance(node, ast.Num):
            pass
        else:
            for child in ast.iter_child_nodes(node):  
                check_div_mult(child, alias_dict) 


    expr = ast.parse(prepare_formula(formula), mode='eval')
    alias_dict = {}
    check_div_mult(expr, alias_dict)
    new_formula = ''.join([i[0] for i in create_tree(expr)])

    aliases = list(string.ascii_lowercase)

    for key in alias_dict.keys():
        alias_dict[key] = aliases.pop(0)
        new_formula = new_formula.replace(key, '<'+ str(alias_dict[key]) + '>')
    
    print(colored('\nNew formula', 'blue', attrs=['dark']))
    print(new_formula, end = '\n\n')
    print(colored('With following variables:', 'blue', attrs=['dark']))
    for key in alias_dict.keys():
        print(colored('<'+ str(alias_dict[key]) + '>', 'magenta', attrs=['bold']) + ' = ', end = '')
        print(clean_formula(key))

In [53]:
for formula in formulas:
    print(colored('Initial formula', 'blue', attrs=['bold', 'underline']))
    print(formula)
    process(formula)
    print('\n')
    print('_'*120+'\n\n')

Initial formula
100* ((M8001C45 -M8001C156 +M8001C46 -M8001C157 +M8001C47 -M8001C158 +M8001C48 -M8001C159 +M8001C49 -M8001C160 +M8001C50 -M8001C161 +M8001C51 -M8001C162 +M8001C52 -M8001C163 +M8001C53 -M8001C164 +M8001C54 -M8001C165 +M8011C196 -M8011C204))/ ((M8001C45 -M8001C156 +M8001C46 -M8001C157 +M8001C47 -M8001C158 +M8001C48 -M8001C159 +M8001C49 -M8001C160 +M8001C50 -M8001C161 +M8001C51 -M8001C162 +M8001C52 -M8001C163 +M8001C53 -M8001C164 +M8001C54 -M8001C165 +M8011C196 -M8011C204) +(M8001C55 -M8001C166 +M8001C56 -M8001C167 +M8001C57 -M8001C168 +M8001C58 -M8001C169 +M8001C59 -M8001C170 +M8001C60 -M8001C171 +M8001C61 -M8001C172 +M8011C197 -M8011C205) +(M8001C62 -M8001C173 +M8001C63 -M8001C174 +M8001C64 -M8001C175 +M8001C65 -M8001C176 +M8001C66 -M8001C202 +M8001C67 -M8001C203 +M8001C68 -M8001C204 +M8001C69 -M8001C205 +M8001C70 -M8001C206 +M8001C71 -M8001C207 +M8001C72 -M8001C208 +M8001C73 -M8001C209 +M8011C198 -M8011C206) +(M8011C199 -M8011C207))

New formula
(<a>/<b>)

With followin